In [622]:
import pandas as pd
import re
from sklearn.utils import shuffle
import random as rand
import numpy as np
import math
import sklearn.naive_bayes as nB

In [623]:
data = pd.read_csv('games_clean.csv')
random = 24
data

,Metacritic,Indie,Presence,Memory,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,...,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip
0,Very good,False,887.0,256.0,2008.0,False,19.99,0.0,False,0.0,...,True,True,False,False,False,False,12.0,9.0,3.0,7.0
1,Good,True,204071.0,2000.0,2014.0,False,19.99,0.0,False,79.0,...,False,True,False,False,False,False,2.0,1.0,NaN,2.0
2,Good,False,13512.0,2000.0,2016.0,False,14.99,0.0,False,50.0,...,False,True,False,False,False,False,4.0,1.0,NaN,1.0
3,Good,False,997391.0,6000.0,2017.0,False,39.99,0.0,True,141.0,...,False,True,True,True,False,False,39.0,33.0,6.0,19.0
4,Very good,False,75371.0,2000.0,2016.0,False,14.99,0.0,True,64.0,...,True,True,False,True,False,False,113.0,49.0,68.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Good,True,1440.0,4000.0,2016.0,False,6.99,0.0,False,8.0,...,False,True,False,False,False,False,1.0,2.0,NaN,2.0
981,Good,True,1965.0,512.0,2011.0,False,4.99,0.0,True,21.0,...,False,True,False,False,False,False,1.0,1.0,NaN,3.0
982,Good,False,5410.0,128.0,2002.0,False,9.99,0.0,False,0.0,...,False,True,False,False,False,False,8.0,4.0,1.0,3.0
983,Good,False,59368.0,2000.0,2010.0,False,24.99,0.0,True,63.0,...,False,True,False,True,False,False,53.0,22.0,50.0,13.0


In [624]:
for name in data.columns[1:]:
    print("Before", name + '\n', data[data[name].isna()][name])
    data[name].fillna(data[name].mean(), inplace=True)
    print("After", name + '\n', data[data[name].isna()][name])

Before Indie
 Series([], Name: Indie, dtype: bool)
After Indie
 Series([], Name: Indie, dtype: bool)
Before Presence
 Series([], Name: Presence, dtype: float64)
After Presence
 Series([], Name: Presence, dtype: float64)
Before Memory
 5     NaN
12    NaN
21    NaN
40    NaN
51    NaN
       ..
898   NaN
919   NaN
924   NaN
931   NaN
975   NaN
Name: Memory, Length: 100, dtype: float64
After Memory
 Series([], Name: Memory, dtype: float64)
Before ReleaseDate
 522   NaN
700   NaN
756   NaN
861   NaN
Name: ReleaseDate, dtype: float64
After ReleaseDate
 Series([], Name: ReleaseDate, dtype: float64)
Before Soundtrack
 Series([], Name: Soundtrack, dtype: bool)
After Soundtrack
 Series([], Name: Soundtrack, dtype: bool)
Before OriginalCost
 78    NaN
101   NaN
112   NaN
141   NaN
163   NaN
207   NaN
212   NaN
266   NaN
342   NaN
353   NaN
384   NaN
393   NaN
398   NaN
447   NaN
490   NaN
559   NaN
565   NaN
733   NaN
736   NaN
764   NaN
817   NaN
881   NaN
890   NaN
920   NaN
940   NaN
Name: O

In [625]:
data.isna().sum()

Metacritic                      0
Indie                           0
Presence                        0
Memory                          0
ReleaseDate                     0
Soundtrack                      0
OriginalCost                    0
DiscountedCost                  0
Controller                      0
Achievements                    0
Storage                         0
Platform-Mobile                 0
Platform-PC                     0
Platform-Xbox                   0
Platform-PlayStation            0
Platform-Nintendo               0
Platform-Sega                   0
RatingsBreakdown-Recommended    0
RatingsBreakdown-Meh            0
RatingsBreakdown-Exceptional    0
RatingsBreakdown-Skip           0
dtype: int64

In [626]:
Labels = data['Metacritic'].unique()
#Labels = ['Very good', 'Good', 'Bad', 'Very bad']
Labels

array(['Very good', 'Good', 'Bad', 'Very bad'], dtype=object)

In [627]:
from scipy.stats import boxcox

exponential = ['Presence', 'OriginalCost', 'Achievements', 'Storage', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']
lambdas = {}

for name in exponential:
    boc = boxcox(data.loc[:,name].apply(lambda x: x + 1*10**(-10)))
    data.loc[:,name] = boc[0]
    print(name, boc[1])
    lambdas[name] = boc[1] 

Presence 0.21990852622283635
OriginalCost 0.334796531233014
Achievements 0.13271676079289932
Storage 0.12270640311751353
RatingsBreakdown-Recommended 0.06823089092378383
RatingsBreakdown-Meh 0.021279269492762667
RatingsBreakdown-Exceptional 0.04326827691867334
RatingsBreakdown-Skip 0.06685777973971664


In [628]:
for name in data.columns:
    print('\033[1m'+name+'\033[0;0m')
    print(data[name].unique())

22.18866283 77.80781825 15.23562412 11.83785407 42.88607066
 86.5195187  40.82301059 51.07066803 41.54069027 90.4092775  82.10263935
 64.60555553 90.35513041 80.65733428 73.59955118 21.34397495 23.6380926
 20.73575189 26.92993356 12.63516149 52.00283042 83.3954312   9.31586878
 80.72631995 78.49698308 35.5372869  73.61761436 90.36957896 18.92744068
 90.34983274 90.3446592  65.34174405 44.63631013 58.8080091  80.80583483
 13.62934695 90.3712464  20.25088711 20.90880952 11.14063909 84.51178701
 60.5250556  90.34764245 29.29575854 58.31207588 41.99633294 69.49023931
 11.39301071 90.35540153 42.93359403 33.19973136 69.48258995 90.35037507
 90.3607398   9.41058351 12.86256658 90.43501051 67.3698873  57.52855218
 56.17349916 59.70015848 47.87466382 68.06815768 39.56671254 12.87110511
 63.85533054 13.58515077 90.35135541 48.87837258 62.15697703 13.35822882
 90.39328853 13.67316519 90.36797395 65.19563002 53.88004829 12.2629631
 33.07104658 38.37010026 16.29753935 37.15614584 90.3542962  74.69

In [629]:
'''import matplotlib.pyplot as plt
cols = ['Presence', 'Memory', 'ReleaseDate', 'OriginalCost', 'DiscountedCost', 'Achievements', 'Storage', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']

for name in cols:
    fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,5))
    data.boxplot(column=name,ax=axes[0])
    data.hist(column=name, ax=axes[1])
plt.show()'''

"import matplotlib.pyplot as plt\ncols = ['Presence', 'Memory', 'ReleaseDate', 'OriginalCost', 'DiscountedCost', 'Achievements', 'Storage', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']\n\nfor name in cols:\n    fig, axes= plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 4]}, figsize=(9,5))\n    data.boxplot(column=name,ax=axes[0])\n    data.hist(column=name, ax=axes[1])\nplt.show()"

In [630]:
from sklearn import preprocessing

In [631]:
encoder = preprocessing.LabelEncoder()
Labels_encoded = encoder.fit_transform(Labels)
Labels_encoded

array([3, 1, 0, 2])

In [632]:
'''from sklearn.preprocessing import LabelBinarizer

OneHot = LabelBinarizer().fit_transform(data.loc[:,'Metacritic'])
index = [x for x in range(data.shape[0])]
for i in data.index:
    data['Metacritic'].update(pd.Series([OneHot[i]], index=[i])) '''

"from sklearn.preprocessing import LabelBinarizer\n\nOneHot = LabelBinarizer().fit_transform(data.loc[:,'Metacritic'])\nindex = [x for x in range(data.shape[0])]\nfor i in data.index:\n    data['Metacritic'].update(pd.Series([OneHot[i]], index=[i])) "

In [633]:
data_expected = encoder.fit_transform(data['Metacritic'])
#data_expected = data[Labels] 
#data_expected = data['Metacritic']
data_input = data.loc[:, data.columns != 'Metacritic']
#data_input = data.loc[:, data.columns.difference(Labels)]

In [634]:
from sklearn.model_selection import train_test_split

In [635]:
X_train, X_test, y_train, y_test = train_test_split(data_input, data_expected, test_size=0.3,random_state=404)

In [636]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [637]:
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB(alpha=1)

In [638]:
data['Metacritic'].dtype

dtype('O')

In [639]:
hist = bnb.fit(X_train, y_train)

ValueError: Negative values in data passed to MultinomialNB (input X)

In [617]:
y_pred = bnb.predict(X_test)

In [618]:
encoder.inverse_transform(y_pred)

array(['Very good', 'Very bad', 'Bad', 'Very good', 'Very good',
       'Very bad', 'Very good', 'Very good', 'Very good', 'Very bad',
       'Very good', 'Very good', 'Very bad', 'Very bad', 'Very good',
       'Very bad', 'Very good', 'Very bad', 'Very bad', 'Bad', 'Bad',
       'Very good', 'Bad', 'Bad', 'Very bad', 'Very good', 'Very bad',
       'Very bad', 'Very bad', 'Very bad', 'Very good', 'Good', 'Bad',
       'Very bad', 'Very good', 'Very good', 'Very bad', 'Very good',
       'Very bad', 'Very bad', 'Very bad', 'Very bad', 'Very bad',
       'Very good', 'Very good', 'Bad', 'Good', 'Very good', 'Very good',
       'Bad', 'Good', 'Bad', 'Bad', 'Bad', 'Bad', 'Bad', 'Very bad',
       'Very good', 'Very good', 'Very good', 'Very good', 'Very bad',
       'Very good', 'Very bad', 'Very bad', 'Very good', 'Very good',
       'Very bad', 'Very good', 'Bad', 'Very good', 'Very good', 'Good',
       'Very good', 'Very bad', 'Bad', 'Very bad', 'Bad', 'Very bad',
       'Very bad', 

In [619]:
from sklearn.metrics import log_loss, accuracy_score

In [620]:
accuracy_score(y_test,y_pred)

0.2635135135135135

In [621]:
log_loss(y_test, y_pred)

ValueError: y_true and y_pred contain different number of classes 3, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 3]